## Streamlit Workbook

This is for figuring out the kinks in the Streamlit app

In [1]:
# cd OneDrive/Data Science/Personal_Projects/Sports/UFC_Prediction/notebooks/final_notebooks
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import sqlite3
import seaborn as sns
from matplotlib.pyplot import figure
from bs4 import BeautifulSoup
import time
import requests     # to get images
import shutil       # to save files locally
import datetime
from scipy.stats import norm
import warnings
warnings.filterwarnings('ignore')
from random import randint
import  random
import os
#os.chdir('C:/Users/tmcro/OneDrive/Data Science/Personal_Projects/Sports/UFC_Prediction')
from cmath import nan
from bs4 import BeautifulSoup
import streamlit as st
import pickle
from datetime import datetime
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
os.chdir('C:/Users/Travis/OneDrive/Data Science/Personal_Projects/Sports/UFC_Prediction_V2/')


In [80]:
def get_next_events(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    # get events
    event1 = soup.find('div', class_='c-card-event--result__info')
    event1_txt = soup.find('div', class_='c-card-event--result__info').text
    event1_url = event1.find('a')['href']
    event1_url = 'https://www.ufc.com' + event1_url
    event1_title = event1_txt.split('\n')[1]
    event1_time = event1_txt.split('/')[1]

    data = pd.DataFrame({'event_title': [event1_title], 'event_url': [event1_url], 'event_date': [event1_time]})

    event2 = soup.find('div', class_='c-card-event--result__info').find_next('div', class_='c-card-event--result__info')
    event2_txt = soup.find('div', class_='c-card-event--result__info').find_next('div', class_='c-card-event--result__info').text
    event2_url = event2.find('a')['href']
    event2_url = 'https://www.ufc.com' + event2_url
    event2_title = event2_txt.split('\n')[1]
    event2_time = event2_txt.split('/')[1]


    data = data.append({'event_title': event2_title, 'event_url': event2_url, 'event_date': event2_time}, ignore_index=True)
    
    event3 = soup.find('div', class_='c-card-event--result__info').find_next('div', class_='c-card-event--result__info').find_next('div', class_='c-card-event--result__info')
    event3_txt = soup.find('div', class_='c-card-event--result__info').find_next('div', class_='c-card-event--result__info').find_next('div', class_='c-card-event--result__info').text
    event3_url = event3.find('a')['href']
    event3_url = 'https://www.ufc.com' + event3_url
    event3_title = event3_txt.split('\n')[1]
    event3_time = event3_txt.split('/')[1]

    data = data.append({'event_title': event3_title, 'event_url': event3_url, 'event_date': event3_time}, ignore_index=True)
    
    return data

# Function to get the fight card for a given event
def get_event_fights(event_url):
    page = requests.get(event_url)
    soup = BeautifulSoup(page.content, 'html.parser')
    # get main card, fight 1

    mcn = soup.find_all('li', class_='l-listing__item')
    # get num of mc
    num_mc = len(mcn)
    # for each mc, do the following
    data = pd.DataFrame()
    n = 0
    for i in mcn:
        mc = mcn[n]
        # fight 1
        fighter1= mc.find('div', class_ ='c-listing-fight__corner-name c-listing-fight__corner-name--red').text
        fighter1 = fighter1.replace('\n', ' ')
        fighter1 = fighter1.strip()
        fighter2 = mc.find('div', class_ ='c-listing-fight__corner-name c-listing-fight__corner-name--blue').text
        fighter2 = fighter2.replace('\n', ' ')
        fighter2 = fighter2.strip()
        weightclass = mc.find('div', class_='c-listing-fight__class-text').text
        fighter1_odds = mc.find('span', class_='c-listing-fight__odds').text
        fighter2_odds = mc.find('span', class_='c-listing-fight__odds').find_next('span', class_='c-listing-fight__odds').text
        fighter1_odds = fighter1_odds.replace('\n', '')
        fighter2_odds = fighter2_odds.replace('\n', '')
        # fighter odds to float
        if (fighter1_odds == '-') :
            fighter1_odds = nan
        if (fighter2_odds == '-') :
            fighter2_odds = nan

        data = data.append({'fighter1': fighter1, 'fighter2': fighter2, 'weightclass': weightclass, 
                            'fighter1_odds': fighter1_odds, 'fighter2_odds': fighter2_odds}, ignore_index=True)
        n = n + 1
    return data


def secret_number(event_url):
    # if no driver open, open one
    driver = None
    if (driver == None):
        driver = webdriver.Chrome('C:\\Users\\Travis\\OneDrive\\Data Science\\Personal_Projects\\Sports\\UFC_Prediction_V2\\chromedriver.exe')
    else:
        driver = driver
    
    driver.get(event_url)
    time.sleep(3)
    # click the first matchup
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    pretty = soup.prettify()
    # find first data-fmid to get first matchup
    fmid_start = pretty.find('data-fmid')
    fmid = pretty[fmid_start+11:fmid_start+16]
    driver.get(event_url +'#' + fmid)
    time.sleep(6)
    # find all links within page
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    # find all iframe src
    iframe = soup.find_all('iframe')
    # find all links
    iframe_text = str(iframe)
    matchup = iframe_text.find('matchup')
    matchup_url = iframe_text[matchup+8:matchup+12]
    print('matchup_url: ' + matchup_url)
    secret_number = matchup_url
    return secret_number


# get next events if event fighter data is not na
def get_next_events2(url):
    data = get_next_events(url)
    for i in range(0, len(data)):
        event_url = data['event_url'][i]
        event_fights = get_event_fights(event_url)
        if (len(event_fights) == 0):
            data = data.drop(i)
    return data

def get_next_event_ufcstats():
    url = 'http://www.ufcstats.com/statistics/events/upcoming'
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    # get events
    event1 = soup.find('td', class_='b-statistics__table-col')
    event1_txt = soup.find('td', class_='b-statistics__table-col').text
    event_txt = event1_txt.replace('   ', '').replace('\n', '').strip()
    event_title = event_txt.split('  ')[0]
    event_date = event_txt.split('  ')[1]
    event1_url = event1.find('a')['href']
    data = pd.DataFrame({'event_title': [event_title], 'event_url': [event1_url], 'event_date': [event_date]})
    return data

def get_fighter_urls(event_details_url):
    page = requests.get(event_details_url)
    soup = BeautifulSoup(page.content, 'html.parser')
    # get events
    events = soup.find_all('tr', class_='b-fight-details__table-row b-fight-details__table-row__hover js-fight-details-click')
    n = 0
    next_event_data = pd.DataFrame()

    for event in events:
        fighters = events[n].find_all('p', class_='b-fight-details__table-text')
        fighter1 = fighters[0].text
        fighter1 = fighter1.replace('  ', '').replace('\n', '').strip()
        fighter2 = fighters[1].text
        fighter2 = fighter2.replace('  ', '').replace('\n', '').strip()
        fighter1_url = fighters[0].find('a')['href']
        fighter2_url = fighters[1].find('a')['href']
        next_event_data = next_event_data.append({'fighter1' :fighter1, 'fighter2:' : fighter2, 'fighter1_url': fighter1_url, 'fighter2_url':fighter2_url, 'fight#' : n+1}, ignore_index = True)
        n += 1

    return next_event_data



# check if it is a saturday
def is_saturday():
    today = str(datetime.today().weekday())
    if today == '5':
        return True
    else:
        return False

# if it is a saturday, get the current event, otherwise, get the next event


next_eventz = get_next_event_ufcstats()

# print the day of the week
next_eventz

,event_title,event_url,event_date
0,UFC 283: Teixeira vs. Hill,http://www.ufcstats.com/event-details/5717efc6...,"January 21, 2023"


In [87]:


##########           DATA        ################
#data = pd.read_csv(home + '/final/aggregates/Double_Fights_DF_V14.csv')

##########           GET EVENTS       ################

# make sure events have fight info. If not, disregard that event
next = get_next_events2('https://www.ufc.com/events')

########           Select Next Event    ################

event = next['event_title'][0]
selected_event = event
event_url =  next['event_url'][next['event_title'] == selected_event].values[0]
selected_event_secret_number = secret_number(event_url)

next_event = get_event_fights(event_url)
fight = st.sidebar.selectbox('Select Fight', next_event['fighter1'] + ' vs. ' + next_event['fighter2'])

## Get Names ##

selected_fighter_1 = fight.split(' vs. ')[0]
selected_fighter_2 = fight.split(' vs. ')[1].strip()


########          Scrape UFC.com Data    ################

# get the matchup fight numbers

page = requests.get(event_url)
soup = BeautifulSoup(page.content, 'html.parser')
h = soup.find_all('div', class_='c-listing-fight')
data_fmid = []
for i in h:
    data_fmid.append(i['data-fmid'])

next_event['fight_number'] = data_fmid[:len(next_event)]
next_event['matchup_url'] = event_url +'#' + next_event['fight_number'].astype(str)
selected_matchup_url = next_event['matchup_url'][next_event['fighter1'] == selected_fighter_1].values[0]

#st.write(' FOR EDIT -- selected_matchup_url: ' + selected_matchup_url)
selected_matchup_url

matchup_url: 1127


'https://www.ufc.com/event/ufc-fight-night-january-14-2023#10315'

In [84]:
# Function to scrape UFC fight data
def grab_matchup_data(matchup_url):
    response = requests.get(matchup_url)
    soup = BeautifulSoup(response.text, 'html.parser').text
    soup = soup.replace('   ', '').replace('\n', '')
    
    # get img src
    

    od = soup.find('Odds')
    rec = soup.find('Record')
    a_record = soup[od + 5 : rec - 2]
    last_fight = soup.find("Last Fight")
    b_record = soup[rec + 7 : last_fight - 5]

    hite = soup.find('Height')
    f = soup.find("' ")
    a_height = soup[f -1 : hite - 2]
    # find second occurance of f
    f2 = soup.find("' ", f + 1)
    b_height = soup[hite + 7 : f2+5]

    # Find reach
    reach = soup.find('Reach')
    # find second occurance of "LB"
    lb = soup.find('LB')
    lb2 = soup.find('LB', lb + 1)
    a_reach = soup[lb2 +5 : reach ]
    inn = soup.find("in ")
    # get the word after reach
    big_space = soup.find('  ', reach + 1)
    b_reach = soup[reach + 6 : big_space]

    # Find Leg Reach
    leg = soup.find('Leg Reach')
    big_space2 = soup.find('  ', big_space + 1)
    a_leg = soup[big_space2 + 2 : leg]
    big_space4 = soup.find('  ', big_space2 + 2)
    b_leg = soup[leg + 10 : leg + 17]

    a_record = a_record.strip()
    b_record = b_record.strip()

    a_height_ft = float(a_height[:1])
    a_height_in = float(a_height[3:].replace("'", "").replace('"', ''))
    a_height = (a_height_ft * 12) + a_height_in 


    b_height_ft = float(b_height[:1])
    b_height_in = float(b_height[3:].replace("'", "").replace('"', ''))
    b_height = (b_height_ft * 12) + b_height_in

    a_reach = float(a_reach.replace(' in', '').strip())
    b_reach = float(b_reach.replace(' in', '').strip())

    a_leg = float(a_leg.replace(' in', '').strip())
    b_leg = float(b_leg.replace(' in', '').strip())

    
    return a_record, b_record, a_height, b_height, a_reach, b_reach, a_leg, b_leg, img_url

url = 'https://www.ufc.com/matchup/' + selected_event_secret_number + '/' + next_event[next_event['fighter1'] == selected_fighter_1]['fight_number'].values[0]

a_record, b_record, a_height, b_height, a_reach, b_reach, a_leg, b_leg, img_url = grab_matchup_data(url)

In [85]:
a_record

'25-5-0'

In [86]:
img_url

-1

In [91]:
driver = webdriver.Chrome()

In [100]:
fighter_last_name = selected_fighter_1.split(' ')[1]
fighter_last_name = fighter_last_name.upper()
fighter_last_name

'STRICKLAND'

In [101]:

driver.get(selected_matchup_url)
    
# wait 10 seconds
time.sleep(3)
soup = BeautifulSoup(page.content, 'html.parser')
h = soup.find_all('img')
imgs = []
for i in h:
    imgs.append(i['src'])

# keep imgs with full_body
imgs = [i for i in imgs if 'full_body' in i]

fighter_img = [i for i in imgs if fighter_last_name in i][0]
fighter_img


'https://dmxg5wxfqgb4u.cloudfront.net/styles/athlete_detail_stance_thumbnail_full_body/s3/2023-01/STRICKLAND_SEAN_L_12-17.png?itok=U-pjxH3U'

In [103]:
# in fighter_img, replace athlete_detail_stance_thumbnail_full_body with athlete_matchup_stats_full_body
fighter_img = fighter_img.replace('athlete_detail_stance_thumbnail_full_body', 'athlete_matchup_stats_full_body')
# end fighter_img at .png
fighter_img = fighter_img[:fighter_img.find('.png') + 4]
fighter_img

'https://dmxg5wxfqgb4u.cloudfront.net/styles/athlete_matchup_stats_full_body/s3/2023-01/STRICKLAND_SEAN_L_12-17.png'